In [14]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

In [4]:
metadata = pd.read_csv("C:\Maya\CS\AnomalyDetectionMicrobiome\Human_Microbiome_Project_1_6_2017_metadata.tab",sep='\t')

In [5]:
metadata.head()

,#SampleID,BarcodeSequence,LinkerPrimerSequence,run_prefix,body_habitat,body_product,body_site,bodysite,dna_extracted,elevation,...,sample_type,scientific_name,sequencecenter,taxon_id,title,qiita_study_title,qiita_study_alias,qiita_owner,qiita_principal_investigator,Description
0,1927.SRS055168.SRX020678.SRR048038,TCAGACTCCTC,ATTACCGCGGCTGCTGG,SRR048038,UBERON:skin,UBERON:sebum,UBERON:skin of elbow,R_Antecubital_fossa,True,97.0,...,Skin,human skin metagenome,BCM,539655,"Structure, function and diversity of the healt...","Structure, function and diversity of the healt...",HMP_V13,Gail Ackermann,Curtis Huttenhower,HMP_SRS055168.SRX020678.SRR048038
1,1927.SRS015346.SRX020577.SRR044088,TCAGAACCAGC,ATTACCGCGGCTGCTGG,SRR044088,UBERON:vagina,UBERON:mucus,UBERON:central vagina,Mid_vagina,True,97.0,...,Vaginal,human vaginal metagenome,WUGSC,1632839,"Structure, function and diversity of the healt...","Structure, function and diversity of the healt...",HMP_V13,Gail Ackermann,Curtis Huttenhower,HMP_SRS015346.SRX020577.SRR044088
2,1927.SRS021613.SRX020675.SRR047757,TCAGAGCTTC,ATTACCGCGGCTGCTGG,SRR047757,UBERON:oral cavity,UBERON:saliva,UBERON:tongue,Tongue_dorsum,True,97.0,...,Oral,human oral metagenome,BCM,447426,"Structure, function and diversity of the healt...","Structure, function and diversity of the healt...",HMP_V13,Gail Ackermann,Curtis Huttenhower,HMP_SRS021613.SRX020675.SRR047757
3,1927.SRS022512.SRX020678.SRR048023,TCAGTATCAAC,ATTACCGCGGCTGCTGG,SRR048023,UBERON:skin,UBERON:sebum,UBERON:skin of elbow,L_Antecubital_fossa,True,97.0,...,Skin,human skin metagenome,BCM,539655,"Structure, function and diversity of the healt...","Structure, function and diversity of the healt...",HMP_V13,Gail Ackermann,Curtis Huttenhower,HMP_SRS022512.SRX020678.SRR048023
4,1927.SRS016305.SRX020525.SRR044526,TCAGCCACGGTC,ATTACCGCGGCTGCTGG,SRR044526,UBERON:skin,UBERON:sebum,UBERON:skin of external ear,L_Retroauricular_crease,True,97.0,...,Skin,human skin metagenome,WUGSC,539655,"Structure, function and diversity of the healt...","Structure, function and diversity of the healt...",HMP_V13,Gail Ackermann,Curtis Huttenhower,HMP_SRS016305.SRX020525.SRR044526


In [6]:
data = pd.read_csv("C:\Maya\CS\AnomalyDetectionMicrobiome\Human_Microbiome_Project_1_6_2017_data.tab",sep='\t')

In [7]:
data.head()

,#OTU ID,1927.SRS018553.SRX020525.SRR044566,1927.SRS047327.SRX020568.SRR047012,1927.SRS065711.SRX022228.SRR058091,1927.SRS011293.SRX020570.SRR045451,1927.SRS014659.SRX020511.SRR045649,1927.SRS063715.SRX022228.SRR058091,1927.SRS062799.SRX022228.SRR058091,1927.SRS015409.SRX020546.SRR043657,1927.SRS014554.SRX020546.SRR043688,...,1928.SRS042698.SRX020530.SRR045847,1928.SRS023827.SRX021470.SRR052699,1928.SRS022769.SRX022226.SRR058089,1928.SRS024716.SRX021470.SRR052699,1928.SRS045191.SRX021470.SRR052699,1928.SRS045171.SRX021470.SRR052699,1928.SRS024704.SRX021470.SRR052699,1928.SRS020726.SRX022239.SRR058109,1928.SRS022067.SRX022231.SRR058120,1928.SRS023274.SRX022223.SRR058086
0,4376890,6.0,7.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,188753,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,114076,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,253429,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4467992,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### extract relevant samples

In [8]:
filtered_data_with_OTUs = data.loc[:, data.sum() > 3000] #5728 samples, 13268 features
filtered_data = filtered_data_with_OTUs.iloc[:, 1:]
filtered_data

,1927.SRS018553.SRX020525.SRR044566,1927.SRS047327.SRX020568.SRR047012,1927.SRS065711.SRX022228.SRR058091,1927.SRS011293.SRX020570.SRR045451,1927.SRS014659.SRX020511.SRR045649,1927.SRS063715.SRX022228.SRR058091,1927.SRS062799.SRX022228.SRR058091,1927.SRS015409.SRX020546.SRR043657,1927.SRS014554.SRX020546.SRR043688,1927.SRS014913.SRX020546.SRR043697,...,1928.SRS042246.SRX020686.SRR048627,1928.SRS057075.SRX022222.SRR058085,1928.SRS043338.SRX022227.SRR058090,1928.SRS022777.SRX022226.SRR058089,1928.SRS016730.SRX019682.SRR040947,1928.SRS021283.SRX022235.SRR058095,1928.SRS018288.SRX019687.SRR041227,1928.SRS016559.SRX019636.SRR040605,1928.SRS016559.SRX019635.SRR040662,1928.SRS018292.SRX019687.SRR041262
0,6.0,7.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13263,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13264,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13265,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13266,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### relative abundance - calc sum

In [9]:
final_data = filtered_data.copy()

for colname in filtered_data.columns:
    final_data.loc[:,colname] = filtered_data[colname] / sum(filtered_data[colname])

final_data.head()

,1927.SRS018553.SRX020525.SRR044566,1927.SRS047327.SRX020568.SRR047012,1927.SRS065711.SRX022228.SRR058091,1927.SRS011293.SRX020570.SRR045451,1927.SRS014659.SRX020511.SRR045649,1927.SRS063715.SRX022228.SRR058091,1927.SRS062799.SRX022228.SRR058091,1927.SRS015409.SRX020546.SRR043657,1927.SRS014554.SRX020546.SRR043688,1927.SRS014913.SRX020546.SRR043697,...,1928.SRS042246.SRX020686.SRR048627,1928.SRS057075.SRX022222.SRR058085,1928.SRS043338.SRX022227.SRR058090,1928.SRS022777.SRX022226.SRR058089,1928.SRS016730.SRX019682.SRR040947,1928.SRS021283.SRX022235.SRR058095,1928.SRS018288.SRX019687.SRR041227,1928.SRS016559.SRX019636.SRR040605,1928.SRS016559.SRX019635.SRR040662,1928.SRS018292.SRX019687.SRR041262
0,0.001055,0.001894,0.000218,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.000000,0.000000,0.000047,0.000213,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000248,0.000000,0.000000,0.00000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001515,0.000000,0.00000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000164,0.00024,0.000265,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Isolation Forest
data for IF

In [10]:
all_feces = metadata[metadata['body_site'] == 'UBERON:feces']['#SampleID']
all_skin_of_nose = metadata[metadata['body_site'] == 'UBERON:skin of nose']['#SampleID']

feces = [sample for sample in all_feces if sample in final_data.columns]
skin_of_nose = [sample for sample in all_skin_of_nose if sample in  final_data.columns]

feces_data = final_data[feces] #418 samples

skin_of_nose_data = final_data[skin_of_nose] #203 samples
few_skin_of_nose_data = skin_of_nose_data.iloc[: ,-42:] #42 samples

inputIF = pd.concat([feces_data, few_skin_of_nose_data], axis=1) #42 samples out of 460 should be outliers

### IF model

In [15]:
model = IsolationForest(n_estimators=100)
model.fit(inputIF.transpose())  # fit 100 trees
#scores = model.score_samples(inputIF.transpose()) # bigger score is more abnirmal
 
scores = model.decision_function(inputIF.transpose()) # bigger score is more normal
sortedScores = sorted(scores)

# indices of anomalies (skin of nose) in sorted scores. higher score = more normal !
anomalies_indices_in_sorted = [i for i in range(len(sortedScores)) if sortedScores[i] in scores[-42:] ]

print(anomalies_indices_in_sorted)

results = [1 for i in range(418)]
more_res = [-1 for i in range(42)]
y = results + more_res

clf = LogisticRegression(solver="liblinear", random_state=0).fit(inputIF.transpose(), y)
# calculate area under the curve
roc_auc_score(y, clf.decision_function(inputIF.transpose()))



[50, 59, 69, 73, 90, 127, 158, 160, 179, 191, 206, 221, 222, 241, 250, 265, 266, 275, 298, 301, 309, 325, 326, 346, 361, 380, 391, 392, 396, 401, 404, 413, 415, 420, 422, 435, 440, 443, 444, 451, 452, 453, 457, 458, 459]


0.9972089314194578

In [16]:
'''
for tree_in_forest in model.estimators_:
    export_graphviz(tree_in_forest,
                    feature_names=data["#OTU ID"],
                    filled=True,
                    rounded=True)


    os.system('dot -Tpng tree.dot -o tree.png')
    
from sklearn.tree import export_graphviz
from six import StringIO 
import pydotplus
from PIL import Image

i_tree = 0
dot_data = StringIO()
for tree_in_forest in model.estimators_:#rfc random forest classifier
    if (i_tree ==3):        
        export_graphviz(tree_in_forest, out_file=dot_data)
        graph = pydotplus.graph_from_dot_data(dot_data.getvalue())        
    i_tree = i_tree + 1
Image(graph.create_png())

    
fn=data.feature_names
cn=data.target_names
fig, axes = plt.subplots(nrows = 1,ncols = 5,figsize = (10,2), dpi=900)
for index in range(0, 5):
    tree.plot_tree(rf.estimators_[index],
                   feature_names = fn, 
                   class_names=cn,
                   filled = True,
                   ax = axes[index]);

    axes[index].set_title('Estimator: ' + str(index), fontsize = 11)
fig.savefig('rf_5trees.png')
'''

'''
estimator = model.estimators_[5]
from sklearn.tree import export_graphviz
# Export as dot file
dot_file = 'C:/Maya/CS/AnomalyDetectionMicrobiome/tree.dot'
export_graphviz(estimator, out_file=dot_file, 
                feature_names = data["#OTU ID"],
                rounded = True, proportion = False, 
                precision = 2, filled = True)

# Convert to png using system command (requires Graphviz)
png_file = 'C:/Maya/CS/AnomalyDetectionMicrobiome/tree.png'
from subprocess import call
call(['dot', '-Tpng', dot_file, '-o',png_file, '-Gdpi=600'])

# Display in jupyter notebook
from IPython.display import Image
Image(filename = 'tree.png')
'''

'\nestimator = model.estimators_[5]\nfrom sklearn.tree import export_graphviz\n# Export as dot file\ndot_file = \'C:/Maya/CS/AnomalyDetectionMicrobiome/tree.dot\'\nexport_graphviz(estimator, out_file=dot_file, \n                feature_names = data["#OTU ID"],\n                rounded = True, proportion = False, \n                precision = 2, filled = True)\n\n# Convert to png using system command (requires Graphviz)\npng_file = \'C:/Maya/CS/AnomalyDetectionMicrobiome/tree.png\'\nfrom subprocess import call\ncall([\'dot\', \'-Tpng\', dot_file, \'-o\',png_file, \'-Gdpi=600\'])\n\n# Display in jupyter notebook\nfrom IPython.display import Image\nImage(filename = \'tree.png\')\n'

graph depth of outlier vs normal
mann whitney test SCIPY
different body sites
bootstrap